In [1]:
import numpy as np
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.applications.nasnet import NASNetMobile
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

2023-04-25 14:09:54.020666: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 14:09:54.251504: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-25 14:09:54.251533: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-25 14:09:54.918409: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocess the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [3]:
# Instantiate a NASNetMobile model with pre-trained weights
base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

2023-04-25 14:10:09.960057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-25 14:10:09.960266: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-25 14:10:09.960330: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-04-25 14:10:09.960385: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-04-25 14:10:10.009571: W tensorflow/c

In [5]:
# Add a new final classification layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


Epoch 1/50


2023-04-25 15:03:18.257643: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.


1563/1563 [==============================] - 111s 62ms/step - loss: 1.7107 - accuracy: 0.4064 - val_loss: 1.6240 - val_accuracy: 0.4303
Epoch 2/50
1563/1563 [==============================] - 103s 66ms/step - loss: 1.5036 - accuracy: 0.4682 - val_loss: 1.5949 - val_accuracy: 0.4382
Epoch 3/50
1563/1563 [==============================] - 93s 59ms/step - loss: 1.3975 - accuracy: 0.5063 - val_loss: 1.6297 - val_accuracy: 0.4401
Epoch 4/50
1563/1563 [==============================] - 93s 59ms/step - loss: 1.2794 - accuracy: 0.5485 - val_loss: 1.6801 - val_accuracy: 0.4446
Epoch 5/50
1563/1563 [==============================] - 103s 66ms/step - loss: 1.1582 - accuracy: 0.5915 - val_loss: 1.7436 - val_accuracy: 0.4380
Epoch 6/50
1563/1563 [==============================] - 93s 60ms/step - loss: 1.0304 - accuracy: 0.6381 - val_loss: 1.8734 - val_accuracy: 0.4351
Epoch 7/50
1563/1563 [==============================] - 103s 66ms/step - loss: 0.9141 - accuracy: 0.6757 - val_loss: 2.0116 - val_ac

In [7]:

print(type(model))  
model.summary()

<class 'keras.engine.functional.Functional'>
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 stem_conv1 (Conv2D)            (None, 15, 15, 32)   864         ['input_1[0][0]']                
                                                                                                  
 stem_bn1 (BatchNormalization)  (None, 15, 15, 32)   128         ['stem_conv1[0][0]']             
                                                                                                  
 activation (Activation)        (None, 15, 15, 32)   0           ['stem_bn1[0][0]']               
                                               

 ion)                                                                                             
                                                                                                  
 separable_conv_1_reduction_rig  (None, 8, 8, 11)    1152        ['separable_conv_1_pad_reduction_
 ht3_stem_1 (SeparableConv2D)                                    right3_stem_1[0][0]']            
                                                                                                  
 separable_conv_1_reduction_lef  (None, 8, 8, 11)    220         ['activation_9[0][0]']           
 t4_stem_1 (SeparableConv2D)                                                                      
                                                                                                  
 activation_6 (Activation)      (None, 8, 8, 11)     0           ['separable_conv_1_bn_reduction_r
                                                                 ight2_stem_1[0][0]']             
          

 reduction_concat_stem_1 (Conca  (None, 8, 8, 44)    0           ['reduction_add_2_stem_1[0][0]', 
 tenate)                                                          'reduction_add3_stem_1[0][0]',  
                                                                  'add[0][0]',                    
                                                                  'reduction_add4_stem_1[0][0]']  
                                                                                                  
 adjust_avg_pool_1_stem_2 (Aver  (None, 8, 8, 32)    0           ['adjust_relu_1_stem_2[0][0]']   
 agePooling2D)                                                                                    
                                                                                                  
 adjust_avg_pool_2_stem_2 (Aver  (None, 8, 8, 32)    0           ['cropping2d[0][0]']             
 agePooling2D)                                                                                    
          

                                                                 eft1_stem_2[0][0]',              
                                                                  'separable_conv_2_bn_reduction_r
                                                                 ight1_stem_2[0][0]']             
                                                                                                  
 separable_conv_1_reduction_rig  (None, 4, 4, 22)    1562        ['separable_conv_1_pad_reduction_
 ht2_stem_2 (SeparableConv2D)                                    right2_stem_2[0][0]']            
                                                                                                  
 separable_conv_1_pad_reduction  (None, 11, 11, 22)  0           ['activation_18[0][0]']          
 _right3_stem_2 (ZeroPadding2D)                                                                   
                                                                                                  
 activatio

                                                                                                  
 reduction_add3_stem_2 (Add)    (None, 4, 4, 22)     0           ['reduction_left3_stem_2[0][0]', 
                                                                  'separable_conv_2_bn_reduction_r
                                                                 ight3_stem_2[0][0]']             
                                                                                                  
 add_1 (Add)                    (None, 4, 4, 22)     0           ['reduction_add_2_stem_2[0][0]', 
                                                                  'reduction_left4_stem_2[0][0]'] 
                                                                                                  
 reduction_add4_stem_2 (Add)    (None, 4, 4, 22)     0           ['separable_conv_2_bn_reduction_l
                                                                 eft4_stem_2[0][0]',              
          

...

 _10 (SeparableConv2D)                                                                            
                                                                                                  
 separable_conv_1_normal_left5_  (None, 1, 1, 176)   32560       ['activation_161[0][0]']         
 10 (SeparableConv2D)                                                                             
                                                                                                  
 separable_conv_1_bn_normal_lef  (None, 1, 1, 176)   704         ['separable_conv_1_normal_left1_1
 t1_10 (BatchNormalization)                                      0[0][0]']                        
                                                                                                  
 separable_conv_1_bn_normal_rig  (None, 1, 1, 176)   704         ['separable_conv_1_normal_right1_
 ht1_10 (BatchNormalization)                                     10[0][0]']                       
          

                                                                                                  
 normal_add_3_10 (Add)          (None, 1, 1, 176)    0           ['normal_left3_10[0][0]',        
                                                                  'adjust_bn_10[0][0]']           
                                                                                                  
 normal_add_4_10 (Add)          (None, 1, 1, 176)    0           ['normal_left4_10[0][0]',        
                                                                  'normal_right4_10[0][0]']       
                                                                                                  
 normal_add_5_10 (Add)          (None, 1, 1, 176)    0           ['separable_conv_2_bn_normal_left
                                                                 5_10[0][0]',                     
                                                                  'normal_bn_1_10[0][0]']         
          

                                                                 t2_11[0][0]']                    
                                                                                                  
 activation_174 (Activation)    (None, 1, 1, 176)    0           ['separable_conv_1_bn_normal_left
                                                                 5_11[0][0]']                     
                                                                                                  
 separable_conv_2_normal_left1_  (None, 1, 1, 176)   35376       ['activation_166[0][0]']         
 11 (SeparableConv2D)                                                                             
                                                                                                  
 separable_conv_2_normal_right1  (None, 1, 1, 176)   32560       ['activation_168[0][0]']         
 _11 (SeparableConv2D)                                                                            
          

 normal_bn_1_12 (BatchNormaliza  (None, 1, 1, 176)   704         ['normal_conv_1_12[0][0]']       
 tion)                                                                                            
                                                                                                  
 activation_177 (Activation)    (None, 1, 1, 176)    0           ['normal_bn_1_12[0][0]']         
                                                                                                  
 activation_179 (Activation)    (None, 1, 1, 176)    0           ['adjust_bn_12[0][0]']           
                                                                                                  
 activation_181 (Activation)    (None, 1, 1, 176)    0           ['adjust_bn_12[0][0]']           
                                                                                                  
 activation_183 (Activation)    (None, 1, 1, 176)    0           ['adjust_bn_12[0][0]']           
          

 ht2_12 (BatchNormalization)                                     12[0][0]']                       
                                                                                                  
 normal_left3_12 (AveragePoolin  (None, 1, 1, 176)   0           ['normal_bn_1_12[0][0]']         
 g2D)                                                                                             
                                                                                                  
 normal_left4_12 (AveragePoolin  (None, 1, 1, 176)   0           ['adjust_bn_12[0][0]']           
 g2D)                                                                                             
                                                                                                  
 normal_right4_12 (AveragePooli  (None, 1, 1, 176)   0           ['adjust_bn_12[0][0]']           
 ng2D)                                                                                            
          